In [1]:

%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from FRVRS import nu, fu
from itertools import product
from pandas import DataFrame, Series
from pysan import get_entropy, get_complexity
from pysan.statistics import get_turbulence
import numpy as np

In [3]:

# load data frames
data_frames_dict = nu.load_data_frames(metrics_evaluation_open_world_df='')
logs_df = data_frames_dict['metrics_evaluation_open_world_df']
print(logs_df.shape) # (276926, 109)

Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_df.pkl.
(171766, 126)


In [4]:

# Add back in the orientation scenes and other scenes not in our poster (which doubles the size of the sample)
if nu.pickle_exists('metrics_evaluation_open_world_distance_delta_df'): distance_delta_df = nu.load_object('metrics_evaluation_open_world_distance_delta_df')
else:
    distance_delta_df = fu.get_distance_deltas_data_frame(logs_df)
    
    # Add the agony column
    if 'has_patient_in_agony' not in distance_delta_df.columns:
        distance_delta_df['has_patient_in_agony'] = False
        for (session_uuid, scene_id), idx_df in distance_delta_df.groupby(fu.scene_groupby_columns):
            
            # Get the whole scene history
            mask_series = True
            for cn in fu.scene_groupby_columns: mask_series &= (logs_df[cn] == eval(cn))
            scene_df = logs_df[mask_series]
            
            # Get whether any patient in the scene is in agony
            mask_series = False
            for cn in fu.mood_columns_list: mask_series |= (scene_df[cn] == 'agony')
            
            # Mark the scene in distance delta as agonistic
            if mask_series.any(): distance_delta_df.loc[idx_df.index, 'has_patient_in_agony'] = True
    
    # Add the patient count column
    if 'patient_count' not in distance_delta_df.columns:
        import numpy as np
        distance_delta_df['patient_count'] = np.nan
        for (session_uuid, scene_id), idx_df in distance_delta_df.groupby(fu.scene_groupby_columns):
            
            # Get the whole scene history
            mask_series = True
            for cn in fu.scene_groupby_columns: mask_series &= (logs_df[cn] == eval(cn))
            scene_df = logs_df[mask_series]
            
            # Get patient_count
            distance_delta_df.loc[idx_df.index, 'patient_count'] = fu.get_patient_count(scene_df)
    
    if 'cluster_label' not in distance_delta_df.columns:
        from sklearn.cluster import DBSCAN
        columns_list = ['actual_engagement_distance', 'ideal_engagement_distance']
        X = distance_delta_df[columns_list].values
        
        # Set appropriate parameters for DBSCAN based on what gives 4 clusters
        dbscan = DBSCAN(eps=5, min_samples=1)
        dbscan.fit(X)
        
        # Get cluster labels for each data point
        distance_delta_df['cluster_label'] = dbscan.labels_
    
    nu.store_objects(metrics_evaluation_open_world_distance_delta_df=distance_delta_df)
    nu.save_data_frames(metrics_evaluation_open_world_distance_delta_df=distance_delta_df)
print(distance_delta_df.shape) # (133, 15)
print(sorted(distance_delta_df.columns))

(76, 15)
['actual_distracted_delta', 'actual_engagement_distance', 'actual_ideal_delta', 'adherence_to_salt', 'cluster_label', 'distracted_engagement_distance', 'has_patient_in_agony', 'ideal_engagement_distance', 'last_still_engagement', 'measure_of_distracted_ordering', 'measure_of_ideal_ordering', 'measure_of_right_ordering', 'patient_count', 'scene_id', 'session_uuid']



# Scene Stats Created for Metrics Evaluation Open World

In [5]:

# if nu.pickle_exists('metrics_evaluation_open_world_scene_stats_df'): scene_stats_df = nu.load_object('metrics_evaluation_open_world_scene_stats_df')
# else:

# Get the combo alphabet list
combos_alphabet = sorted(set([sequence_tuple[0] + '-' + sequence_tuple[1] for sequence_tuple in product(fu.patient_sort_order, fu.injury_severity_order)]))

rows_list = []
engagment_columns_list = ['patient_id', 'engagement_start', 'location_tuple', 'patient_sort', 'predicted_priority', 'injury_severity']
for (session_uuid, scene_id), idx_df in distance_delta_df.groupby(fu.scene_groupby_columns):
    row_dict = list(idx_df.T.to_dict().values())[0]
    
    # Get the whole scene history
    mask_series = True
    for cn in fu.scene_groupby_columns: mask_series &= (logs_df[cn] == eval(cn))
    scene_df = logs_df[mask_series]
    
    if scene_df.shape[0]:
        row_dict['participant_id'] = scene_df.participant_id.iloc[0]
        
        # Get the engagement sequence and the stats from that
        actual_engagement_order = fu.get_actual_engagement_order(scene_df, verbose=False)
        if actual_engagement_order:
            df = DataFrame(actual_engagement_order, columns=engagment_columns_list)
            patient_sorts_list = df.apply(lambda row_series: row_series.patient_sort, axis='columns').tolist()
            patient_sorts_sequence, patient_sorts_string_to_integer_map = nu.convert_strings_to_integers(patient_sorts_list, alphabet_list=fu.patient_sort_order)
            injury_severitys_list = df.apply(lambda row_series: row_series.injury_severity, axis='columns').tolist()
            injury_severitys_sequence, injury_severitys_string_to_integer_map = nu.convert_strings_to_integers(
                injury_severitys_list, alphabet_list=fu.injury_severity_order
            )
            combos_list = [str(patient_sort) + '-' + str(injury_severity) for patient_sort, injury_severity in zip(patient_sorts_list, injury_severitys_list)]
            combos_sequence, combos_string_to_integer_map = nu.convert_strings_to_integers(combos_list, alphabet_list=combos_alphabet)
            
            # Replace negative values with 9s to maintain character width
            patient_sorts_sequence[patient_sorts_sequence < 0] = 9
            injury_severitys_sequence[injury_severitys_sequence < 0] = 9
            combos_sequence[combos_sequence < 0] = 9
            
            for fn in [get_turbulence, get_entropy, get_complexity]:
                function_name = fn.__name__
                for cn in ['patient_sort', 'injury_severity', 'combo']:
                    dictionary_key = function_name.replace('get', cn)
                    variable_name = f'{cn}s_sequence'
                    try:
                        dictionary_value = fn(list(eval(variable_name)))
                        row_dict[dictionary_key] = dictionary_value
                    except: continue
        
        # Get all the FRVRS utils scalar scene values
        row_dict['first_engagement'] = fu.get_first_engagement(scene_df)
        row_dict['first_treatment'] = fu.get_first_treatment(scene_df)
        row_dict['injury_correctly_treated_count'] = fu.get_injury_correctly_treated_count(scene_df)
        row_dict['injury_not_treated_count'] = fu.get_injury_not_treated_count(scene_df)
        row_dict['injury_treatments_count'] = fu.get_injury_treatments_count(scene_df)
        row_dict['injury_wrongly_treated_count'] = fu.get_injury_wrongly_treated_count(scene_df)
        row_dict['is_scene_aborted'] = fu.get_is_scene_aborted(scene_df)
        row_dict['last_engagement'] = fu.get_last_engagement(scene_df)
        row_dict['measure_of_right_ordering'] = fu.get_measure_of_right_ordering(scene_df)
        row_dict['patient_count'] = fu.get_patient_count(scene_df)
        row_dict['percent_hemorrhage_controlled'] = fu.get_percent_hemorrhage_controlled(scene_df)
        row_dict['pulse_taken_count'] = fu.get_pulse_taken_count(scene_df)
        row_dict['scene_end'] = fu.get_scene_end(scene_df)
        row_dict['scene_start'] = fu.get_scene_start(scene_df)
        row_dict['scene_type'] = fu.get_scene_type(scene_df)
        row_dict['stills_value'] = fu.get_stills_value(scene_df)
        row_dict['teleport_count'] = fu.get_teleport_count(scene_df)
        row_dict['time_to_hemorrhage_control_per_patient'] = fu.get_time_to_hemorrhage_control_per_patient(scene_df)
        row_dict['time_to_last_hemorrhage_controlled'] = fu.get_time_to_last_hemorrhage_controlled(scene_df)
        row_dict['total_actions'] = fu.get_total_actions_count(scene_df)
        row_dict['triage_time'] = fu.get_triage_time(scene_df)
        row_dict['voice_capture_count'] = fu.get_voice_capture_count(scene_df)
        row_dict['walk_command_count'] = fu.get_walk_command_count(scene_df)
        row_dict['walk_value'] = fu.get_walk_value(scene_df)
        row_dict['walkers_value'] = fu.get_walkers_value(scene_df)
        row_dict['wave_command_count'] = fu.get_wave_command_count(scene_df)
        row_dict['wave_value'] = fu.get_wave_value(scene_df)
    
    rows_list.append(row_dict)
scene_stats_df = DataFrame(rows_list)
nu.store_objects(metrics_evaluation_open_world_scene_stats_df=scene_stats_df)
nu.save_data_frames(metrics_evaluation_open_world_scene_stats_df=scene_stats_df)
print(scene_stats_df.shape) # (43, 49)

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_scene_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_scene_stats_df.csv
(76, 50)


In [6]:

# Check if all the patient IDs in any run are some variant of Mike and designate those runs as "Orientation"
new_column_name = 'scene_type'
if (new_column_name in scene_stats_df.columns): scene_stats_df = scene_stats_df.drop(columns=new_column_name)
if (new_column_name not in scene_stats_df.columns): scene_stats_df[new_column_name] = 'Triage'
column_value = 'Orientation'
if (column_value not in scene_stats_df.scene_type):
    
    # Filter out those files from the dataset and mark them
    for (session_uuid, scene_id), scene_df in logs_df.groupby(fu.scene_groupby_columns):
        patients_list = sorted(scene_df[~scene_df.patient_id.isnull()].patient_id.unique())
        is_mike_series = Series(patients_list).map(lambda x: 'mike' in str(x).lower())
        if is_mike_series.all():
            mask_series = True
            for cn in fu.scene_groupby_columns: mask_series &= (scene_stats_df[cn] == eval(cn))
            scene_stats_df.loc[mask_series, new_column_name] = column_value
    
    # Store the results and show the new data frame shape
    nu.store_objects(metrics_evaluation_open_world_scene_stats_df=scene_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_scene_stats_df=scene_stats_df)
    print(scene_stats_df.shape) # (76, 49)
    
display(scene_stats_df.groupby(['patient_count', 'is_scene_aborted', new_column_name]).size().to_frame().rename(columns={0: 'record_count'}))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_scene_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_scene_stats_df.csv
(76, 50)


record_count
patient_count is_scene_aborted scene_type               
0.0           False            Orientation             1
              True             Orientation            12
1.0           True             Triage                  1
9.0           False            Triage                  1
              True             Triage                  1
10.0          False            Triage                  2
              True             Triage                  2
11.0          False            Triage                 10
              True             Triage                  3
12.0          False            Triage                  5
14.0          False            Triage                  3
              True             Triage                  1
15.0          False            Triage                  3
              True             Triage                  6
16.0          False            Triage                  4
              True             Triage                  5

In [7]:

# Any runs longer than that 16 minutes are probably an instance
# of someone taking off the headset and setting it on the ground.
# 1 second = 1,000 milliseconds; 1 minute = 60 seconds
new_column_name = 'is_scene_aborted'
# if (new_column_name in logs_df.columns): logs_df = logs_df.drop(columns=new_column_name)
if (new_column_name not in scene_stats_df.columns):
    scene_stats_df[new_column_name] = False
    for (session_uuid, scene_id), scene_df in logs_df.groupby(fu.scene_groupby_columns):
        mask_series = True
        for cn in fu.scene_groupby_columns: mask_series &= (scene_stats_df[cn] == eval(cn))
        scene_stats_df.loc[mask_series, new_column_name] = fu.get_is_scene_aborted(scene_df)
    
    # Store the results and show the new data frame shape
    nu.store_objects(metrics_evaluation_open_world_scene_stats_df=scene_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_scene_stats_df=scene_stats_df)
    
    print(scene_stats_df.shape) # (880, 59)

display(scene_stats_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

,record_count
is_scene_aborted,
False,29
True,31



# Verification

In [13]:

mask_series = ~scene_df.patient_id.isnull()
df = scene_df[mask_series]
if df.shape[0]: display(df.sample(min(df.shape[0], 5)).dropna(axis='columns', how='all').T)

,169706,170125,169837,170521,171426
action_type,PLAYER_GAZE,PLAYER_GAZE,PLAYER_GAZE,TOOL_APPLIED,TOOL_APPLIED
action_tick,156782,224504,180541,417177,840385
event_time,2024-03-20 10:26:58,2024-03-20 10:28:06,2024-03-20 10:27:22,2024-03-20 10:31:18,2024-03-20 10:38:21
session_uuid,fc676b00-9559-467d-adbb-696dd6e9fb08,fc676b00-9559-467d-adbb-696dd6e9fb08,fc676b00-9559-467d-adbb-696dd6e9fb08,fc676b00-9559-467d-adbb-696dd6e9fb08,fc676b00-9559-467d-adbb-696dd6e9fb08
file_name,Metrics Evaluation Open World/fc676b00-9559-46...,Metrics Evaluation Open World/fc676b00-9559-46...,Metrics Evaluation Open World/fc676b00-9559-46...,Metrics Evaluation Open World/fc676b00-9559-46...,Metrics Evaluation Open World/fc676b00-9559-46...
logger_version,1.4,1.4,1.4,1.4,1.4
tool_applied_type,NaN,NaN,NaN,Gauze_Dressing,Gauze_Dressing
tool_applied_attachment_point,NaN,NaN,NaN,Tourniquet_Icon (UnityEngine.GameObject),Tourniquet_Icon (UnityEngine.GameObject)
tool_applied_attach_message,NaN,NaN,NaN,AppliedDressingGauze,AppliedDressingGauze
player_gaze_location,"(6.7, 0.0, -0.5)","(4.9, 0.0, 0.4)","(6.7, 0.0, -0.5)",NaN,NaN


In [9]:

# Get them set up as if we were going to run triage accuracy and time to hemorrhage control stats on them
# I know they are small samples, I primarily want you to let us know what information is missing for you to look at those things.
# I am also interested in variability around each patient. Are you able to segregate the files by patient and tell me something about
# how different participants acted (assess, treat, tag) in response to each patient?
# I also want to extract from each csv a list of the actions the responder did (engage, assess, treat, and tag actions, not movement or others)
# so that I can see a list of what they did in the scenario (this will eventually form the basis of delegation decisions).
# Basically I would like you to play with this small sample in anticipation of receiving a similar, larger dataset from these new scenarios.
# Let me know if you have questions but also please be patient as this is low priority compared to evaluation preparation this week.
import os
import glob
import os.path as osp

data_frames_dict = nu.load_data_frames(verbose=False, metrics_evaluation_open_world_scene_stats_df='', metrics_evaluation_open_world_json_stats_df='')
json_stats_df = data_frames_dict['metrics_evaluation_open_world_json_stats_df']
logs_folder = '../data/logs'
pattern = osp.join(logs_folder, 'Disaster Day 3.6.2024 ITM Files 405*')
csv_count = 0
for logs_path in glob.glob(pattern):
    csv_count += len([fn for sub_directory, directories_list, files_list in os.walk(logs_path) for fn in files_list if fn.lower().endswith('.csv')])
uuids_count = json_stats_df.session_uuid.unique().shape[0]
elevens_mask = (scene_stats_df.patient_count >= 11) & (scene_stats_df.is_scene_aborted == False)
scene_type_dict = scene_stats_df[elevens_mask].groupby('scene_type').size().to_dict()
orientation_count = scene_type_dict['Orientation'] if 'Orientation' in scene_type_dict else 0
triage_count = scene_type_dict['Triage'] if 'Triage' in scene_type_dict else 0
aborted_count = scene_stats_df[scene_stats_df.is_scene_aborted].shape[0]
aborted_suffix = 's' if (aborted_count != 1) else ''
one_triage_dict = json_stats_df.groupby('is_a_one_triage_file').size().to_dict()
print(f'''Okay, I have ingested Disaster day 3.6.2024 ITM files 405F.zip and Disaster Day 3.6.2024 ITM Files 405E.zip and have verified that:
•	There are {csv_count} CSV files in the zip file.
•	There are {uuids_count} unique session UUIDs in there.
•	There are {orientation_count} 11-patient-or-above Orientation scenes and {triage_count} Triage scenes in there.
•	The time difference between when the scene starts and the last engagement for {aborted_count} scene{aborted_suffix} is longer than 16 minutes.
•	There are {one_triage_dict.get(True, 0)} files that have one and only one triage scene in them. (The other {one_triage_dict.get(False, 0)}''')
print(''' are not one-triage-scene files).
If any of these is unexpected, please explain why.
''')
mask_series = (scene_stats_df.scene_type == 'Orientation') & elevens_mask
uuids_list = sorted(scene_stats_df[mask_series].session_uuid.unique())
mask_series = json_stats_df.session_uuid.isin(uuids_list)
if mask_series.any():
    print(
        f'P.S. The files with orientation scenes in them are:'
    )
    for file_name in sorted(json_stats_df[mask_series].session_file_name): print(file_name)

Okay, I have ingested Disaster day 3.6.2024 ITM files 405F.zip and Disaster Day 3.6.2024 ITM Files 405E.zip and have verified that:
•	There are 19 CSV files in the zip file.
•	There are 60 unique session UUIDs in there.
•	There are 0 11-patient-or-above Orientation scenes and 31 Triage scenes in there.
•	The time difference between when the scene starts and the last engagement for 33 scenes is longer than 16 minutes.
•	There are 41 files that have one and only one triage scene in them. (The other 19
 are not one-triage-scene files).
If any of these is unexpected, please explain why.



In [10]:

from IPython.display import HTML

if 'is_in_registry' in json_stats_df:
    registry_uuids = sorted(json_stats_df[json_stats_df.is_in_registry].session_uuid.unique())
    print(f'Of the {len(registry_uuids)} files in the registry, ', end='')
    mask_series = scene_stats_df.session_uuid.isin(registry_uuids) & (scene_stats_df.scene_type == 'Triage')
    print(f'of the {scene_stats_df[mask_series].shape[0]} triage scenes in those files, this is the counts of the values of responder type:')
    display(HTML(json_stats_df.responder_type.value_counts().to_frame().to_html()))

Of the 0 files in the registry, of the 0 triage scenes in those files, this is the counts of the values of responder type:


,responder_type


In [11]:

display(HTML(scene_stats_df.groupby(['patient_count', 'scene_type']).size().to_frame().rename(columns={0: 'record_count'}).reset_index(drop=False).to_html()))

,patient_count,scene_type,record_count
0,0,Orientation,13
1,1,Triage,1
2,9,Triage,4
3,10,Triage,10
4,11,Triage,15
5,12,Triage,7
6,14,Triage,5
7,15,Triage,10
8,16,Triage,11


In [14]:

mask_series = (scene_stats_df.scene_type == 'Orientation')
df = scene_stats_df[mask_series].dropna(axis='columns', how='all')
display(df.sample(10).T)
if 'session_uuid' in df.columns:
    session_uuids_list = sorted(df.session_uuid.unique())
    mask_series = json_stats_df.session_uuid.isin(session_uuids_list)
    df = json_stats_df[mask_series].dropna(axis='columns', how='all')
    display(df.sample(9).T)

,62,9,35,33,57,65,31,52,47,55
session_uuid,c6d3a90f-68c0-4948-bd96-537e80973605,21f8cb5d-f5ac-4a01-9287-43df5f6751a1,67dc0230-511d-41ac-ae9b-850900ab9e6a,6666ee6b-863b-49d1-8097-97e6aa4fb39d,aecfcd56-2262-40a8-9bb8-088f57d46f3f,c99de80f-15cc-45cb-aa64-5af0f2f118ca,5d94f0d4-a1b1-4d18-8a62-591e196006a9,a7ce6f7b-6466-4281-9496-92b640d9d04b,922ad146-241a-4ea6-8ff1-413d7e0d16ec,acf74a81-a534-44c7-9cb1-67ec381b5ee0
scene_id,1,1,1,1,1,1,1,1,1,1
actual_engagement_distance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ideal_engagement_distance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
distracted_engagement_distance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
actual_ideal_delta,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
actual_distracted_delta,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
adherence_to_salt,False,False,False,False,False,False,False,False,False,False
has_patient_in_agony,False,False,False,False,False,False,False,False,False,False
patient_count,0,0,0,0,0,0,0,0,0,0


,9,25,24,46,21,56,50,26,16
logger_version,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4
session_uuid,21f8cb5d-f5ac-4a01-9287-43df5f6751a1,6666ee6b-863b-49d1-8097-97e6aa4fb39d,5d94f0d4-a1b1-4d18-8a62-591e196006a9,acf74a81-a534-44c7-9cb1-67ec381b5ee0,50b15e40-9860-4574-8ab8-0bd960fe27de,ddfb4e6e-f7c3-4321-89df-b6208f336318,c6d3a90f-68c0-4948-bd96-537e80973605,67dc0230-511d-41ac-ae9b-850900ab9e6a,3cf14e31-f416-4c78-8a69-91bf0c685448
file_name,Metrics Evaluation Open World/21f8cb5d-f5ac-4a...,Metrics Evaluation Open World/6666ee6b-863b-49...,Metrics Evaluation Open World/5d94f0d4-a1b1-4d...,Metrics Evaluation Open World/acf74a81-a534-44...,Metrics Evaluation Open World/50b15e40-9860-45...,Metrics Evaluation Open World/ddfb4e6e-f7c3-43...,Metrics Evaluation Open World/c6d3a90f-68c0-49...,Metrics Evaluation Open World/67dc0230-511d-41...,Metrics Evaluation Open World/3cf14e31-f416-4c...
session_file_date,2024-03-20 00:00:00,2024-03-20 00:00:00,2024-03-20 00:00:00,2024-03-14 00:00:00,2024-03-20 00:00:00,2024-03-14 00:00:00,2024-03-20 00:00:00,2024-03-14 00:00:00,2024-03-20 00:00:00
encounter_layout,Submarine,Urban,Jungle,Submarine,Submarine,Desert,Submarine,Jungle,Jungle
...,...,...,...,...,...,...,...,...,...
ST_ConfFC_Omni_Text,NaN,NaN,NaN,4.0,NaN,3.0,3.0,2.0,NaN
AD_Del_Text,NaN,NaN,NaN,1.0,NaN,1.0,1.0,0.5,0.0
AD_ConfFC_Text,NaN,NaN,NaN,2.5,NaN,1.5,2.5,2.0,0.0
AD_Del_Omni_Text,NaN,NaN,NaN,1.0,NaN,1.0,0.0,1.0,NaN



----

In [8]:

print()
is_scalar_lambda = lambda value: isinstance(value, (int, float, str, bool, np.number))
for (session_uuid, scene_id), idx_df in distance_delta_df.groupby(fu.scene_groupby_columns):
    
    # Get the whole scene history
    mask_series = True
    for cn in fu.scene_groupby_columns: mask_series &= (logs_df[cn] == eval(cn))
    scene_df = logs_df[mask_series]
    
    if is_scalar_lambda(fu.get_scene_start(scene_df)): print(
        "    row_dict['scene_start'] = fu.get_scene_start(scene_df)"
    )
    if is_scalar_lambda(fu.get_last_engagement(scene_df)): print(
        "    row_dict['last_engagement'] = fu.get_last_engagement(scene_df)"
    )
    if is_scalar_lambda(fu.get_player_location(scene_df, action_tick=0)): print(
        "    row_dict['player_location'] = fu.get_player_location(scene_df, action_tick)"
    )
    if is_scalar_lambda(fu.get_scene_type(scene_df)): print(
        "    row_dict['scene_type'] = fu.get_scene_type(scene_df)"
    )
    if is_scalar_lambda(fu.get_scene_end(scene_df)): print(
        "    row_dict['scene_end'] = fu.get_scene_end(scene_df)"
    )
    if is_scalar_lambda(fu.get_patient_count(scene_df)): print(
        "    row_dict['patient_count'] = fu.get_patient_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_injury_treatments_count(scene_df)): print(
        "    row_dict['injury_treatments_count'] = fu.get_injury_treatments_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_injury_not_treated_count(scene_df)): print(
        "    row_dict['injury_not_treated_count'] = fu.get_injury_not_treated_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_injury_correctly_treated_count(scene_df)): print(
        "    row_dict['injury_correctly_treated_count'] = fu.get_injury_correctly_treated_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_injury_wrongly_treated_count(scene_df)): print(
        "    row_dict['injury_wrongly_treated_count'] = fu.get_injury_wrongly_treated_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_pulse_taken_count(scene_df)): print(
        "    row_dict['pulse_taken_count'] = fu.get_pulse_taken_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_teleport_count(scene_df)): print(
        "    row_dict['teleport_count'] = fu.get_teleport_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_voice_capture_count(scene_df)): print(
        "    row_dict['voice_capture_count'] = fu.get_voice_capture_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_walk_command_count(scene_df)): print(
        "    row_dict['walk_command_count'] = fu.get_walk_command_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_wave_command_count(scene_df)): print(
        "    row_dict['wave_command_count'] = fu.get_wave_command_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_first_engagement(scene_df)): print(
        "    row_dict['first_engagement'] = fu.get_first_engagement(scene_df)"
    )
    if is_scalar_lambda(fu.get_first_treatment(scene_df)): print(
        "    row_dict['first_treatment'] = fu.get_first_treatment(scene_df)"
    )
    if is_scalar_lambda(fu.get_ideal_engagement_order(scene_df, tuples_list=None)): print(
        "    row_dict['ideal_engagement_order'] = fu.get_ideal_engagement_order(scene_df, tuples_list=None)"
    )
    if is_scalar_lambda(fu.get_is_scene_aborted(scene_df)): print(
        "    row_dict['is_scene_aborted'] = fu.get_is_scene_aborted(scene_df)"
    )
    if is_scalar_lambda(fu.get_triage_time(scene_df)): print(
        "    row_dict['triage_time'] = fu.get_triage_time(scene_df)"
    )
    if is_scalar_lambda(fu.get_dead_patients(scene_df)): print(
        "    row_dict['dead_patients'] = fu.get_dead_patients(scene_df)"
    )
    if is_scalar_lambda(fu.get_still_patients(scene_df)): print(
        "    row_dict['still_patients'] = fu.get_still_patients(scene_df)"
    )
    if is_scalar_lambda(fu.get_total_actions_count(scene_df)): print(
        "    row_dict['total_actions'] = fu.get_total_actions_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_actual_and_ideal_patient_sort_sequences(scene_df)): print(
        "    row_dict['actual_and_ideal_sequences'] = fu.get_actual_and_ideal_patient_sort_sequences(scene_df)"
    )
    if is_scalar_lambda(fu.get_measure_of_right_ordering(scene_df)): print(
        "    row_dict['measure_of_right_ordering'] = fu.get_measure_of_right_ordering(scene_df)"
    )
    if is_scalar_lambda(fu.get_percent_hemorrhage_controlled(scene_df)): print(
        "    row_dict['percent_hemorrhage_controlled'] = fu.get_percent_hemorrhage_controlled(scene_df)"
    )
    if is_scalar_lambda(fu.get_time_to_last_hemorrhage_controlled(scene_df)): print(
        "    row_dict['time_to_last_hemorrhage_controlled'] = fu.get_time_to_last_hemorrhage_controlled(scene_df)"
    )
    if is_scalar_lambda(fu.get_time_to_hemorrhage_control_per_patient(scene_df)): print(
        "    row_dict['time_to_hemorrhage_control_per_patient'] = fu.get_time_to_hemorrhage_control_per_patient(scene_df)"
    )
    if is_scalar_lambda(fu.get_triage_priority_data_frame(scene_df)): print(
        "    row_dict['triage_priority_data_frame'] = fu.get_triage_priority_data_frame(scene_df)"
    )
    if is_scalar_lambda(fu.get_actual_engagement_order(scene_df)): print(
        "    row_dict['actual_engagement_order'] = fu.get_actual_engagement_order(scene_df)"
    )
    if is_scalar_lambda(fu.get_distracted_engagement_order(scene_df, tuples_list=None)): print(
        "    row_dict['distracted_engagement_order'] = fu.get_distracted_engagement_order(scene_df, tuples_list=None)"
    )
    if is_scalar_lambda(fu.get_stills_value(scene_df)): print(
        "    row_dict['stills_value'] = fu.get_stills_value(scene_df)"
    )
    if is_scalar_lambda(fu.get_walkers_value(scene_df)): print(
        "    row_dict['walkers_value'] = fu.get_walkers_value(scene_df)"
    )
    if is_scalar_lambda(fu.get_wave_value(scene_df)): print(
        "    row_dict['wave_value'] = fu.get_wave_value(scene_df)"
    )
    if is_scalar_lambda(fu.get_walk_value(scene_df)): print(
        "    row_dict['walk_value'] = fu.get_walk_value(scene_df)"
    )
    break


    row_dict['scene_start'] = fu.get_scene_start(scene_df)
    row_dict['last_engagement'] = fu.get_last_engagement(scene_df)
    row_dict['scene_type'] = fu.get_scene_type(scene_df)
    row_dict['scene_end'] = fu.get_scene_end(scene_df)
    row_dict['patient_count'] = fu.get_patient_count(scene_df)
    row_dict['injury_treatments_count'] = fu.get_injury_treatments_count(scene_df)
    row_dict['injury_not_treated_count'] = fu.get_injury_not_treated_count(scene_df)
    row_dict['injury_correctly_treated_count'] = fu.get_injury_correctly_treated_count(scene_df)
    row_dict['injury_wrongly_treated_count'] = fu.get_injury_wrongly_treated_count(scene_df)
    row_dict['pulse_taken_count'] = fu.get_pulse_taken_count(scene_df)
    row_dict['teleport_count'] = fu.get_teleport_count(scene_df)
    row_dict['voice_capture_count'] = fu.get_voice_capture_count(scene_df)
    row_dict['walk_command_count'] = fu.get_walk_command_count(scene_df)
    row_dict['wave_command_count'] = fu.get_wave_command